# Aufmerksamkeitsmechanismen und Transformer

Ein wesentlicher Nachteil von rekurrenten Netzwerken ist, dass alle Wörter in einer Sequenz den gleichen Einfluss auf das Ergebnis haben. Dies führt zu suboptimaler Leistung bei Standard-LSTM-Encoder-Decoder-Modellen für Sequenz-zu-Sequenz-Aufgaben wie der Erkennung benannter Entitäten (Named Entity Recognition) und maschineller Übersetzung. In der Realität haben bestimmte Wörter in der Eingabesequenz oft mehr Einfluss auf die sequentiellen Ausgaben als andere.

Betrachten wir ein Sequenz-zu-Sequenz-Modell, wie es bei der maschinellen Übersetzung verwendet wird. Es wird durch zwei rekurrente Netzwerke implementiert, wobei ein Netzwerk (**Encoder**) die Eingabesequenz in einen versteckten Zustand komprimiert und ein anderes Netzwerk (**Decoder**) diesen versteckten Zustand in das übersetzte Ergebnis entfaltet. Das Problem bei diesem Ansatz ist, dass der Endzustand des Netzwerks Schwierigkeiten hat, sich an den Anfang eines Satzes zu erinnern, was zu einer schlechten Modellqualität bei langen Sätzen führt.

**Aufmerksamkeitsmechanismen** bieten eine Möglichkeit, den kontextuellen Einfluss jedes Eingabevektors auf jede Ausgabewahrscheinlichkeit des RNN zu gewichten. Dies wird durch die Erstellung von Abkürzungen zwischen den Zwischenzuständen des Eingabe-RNN und des Ausgabe-RNN umgesetzt. Auf diese Weise berücksichtigen wir bei der Generierung des Ausgabesymbols $y_t$ alle versteckten Eingabezustände $h_i$, mit unterschiedlichen Gewichtungskoeffizienten $\alpha_{t,i}$.

![Bild eines Encoder/Decoder-Modells mit einer additiven Aufmerksamkeits-Schicht](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Das Encoder-Decoder-Modell mit additivem Aufmerksamkeitsmechanismus aus [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), zitiert aus [diesem Blogpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Die Aufmerksamkeitsmatrix $\{\alpha_{i,j}\}$ repräsentiert den Grad, in dem bestimmte Eingabewörter bei der Generierung eines bestimmten Wortes in der Ausgabesequenz eine Rolle spielen. Unten ist ein Beispiel für eine solche Matrix dargestellt:

![Bild eines Beispiel-Alignments, gefunden von RNNsearch-50, aus Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Abbildung aus [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Aufmerksamkeitsmechanismen sind für einen Großteil des aktuellen oder nahezu aktuellen Stands der Technik in der Verarbeitung natürlicher Sprache verantwortlich. Das Hinzufügen von Aufmerksamkeit erhöht jedoch die Anzahl der Modellparameter erheblich, was zu Skalierungsproblemen bei RNNs führte. Eine zentrale Einschränkung bei der Skalierung von RNNs ist, dass die rekurrente Natur der Modelle es schwierig macht, das Training zu batchen und zu parallelisieren. In einem RNN muss jedes Element einer Sequenz in der Reihenfolge verarbeitet werden, was bedeutet, dass es nicht leicht parallelisiert werden kann.

Die Einführung von Aufmerksamkeitsmechanismen in Kombination mit dieser Einschränkung führte zur Entwicklung der heute bekannten und genutzten Transformer-Modelle, die den Stand der Technik darstellen, von BERT bis OpenGPT3.

## Transformer-Modelle

Anstatt den Kontext jeder vorherigen Vorhersage in den nächsten Evaluierungsschritt weiterzuleiten, verwenden **Transformer-Modelle** **Positionskodierungen** und Aufmerksamkeit, um den Kontext einer gegebenen Eingabe innerhalb eines bereitgestellten Textfensters zu erfassen. Das folgende Bild zeigt, wie Positionskodierungen mit Aufmerksamkeit den Kontext innerhalb eines bestimmten Fensters erfassen können.

![Animiertes GIF, das zeigt, wie die Evaluierungen in Transformer-Modellen durchgeführt werden.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Da jede Eingabeposition unabhängig auf jede Ausgabeposition abgebildet wird, können Transformer besser parallelisiert werden als RNNs, was viel größere und ausdrucksstärkere Sprachmodelle ermöglicht. Jeder Aufmerksamkeitskopf kann verwendet werden, um unterschiedliche Beziehungen zwischen Wörtern zu lernen, was die nachgelagerten Aufgaben der Verarbeitung natürlicher Sprache verbessert.

**BERT** (Bidirectional Encoder Representations from Transformers) ist ein sehr großes mehrschichtiges Transformer-Netzwerk mit 12 Schichten für *BERT-base* und 24 für *BERT-large*. Das Modell wird zunächst auf einem großen Textkorpus (Wikipedia + Bücher) mit unüberwachtem Training (Vorhersage maskierter Wörter in einem Satz) vortrainiert. Während des Vortrainings erwirbt das Modell ein signifikantes Maß an Sprachverständnis, das dann mit anderen Datensätzen durch Feintuning genutzt werden kann. Dieser Prozess wird als **Transferlernen** bezeichnet.

![Bild von http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Es gibt viele Variationen von Transformer-Architekturen, darunter BERT, DistilBERT, BigBird, OpenGPT3 und mehr, die feinabgestimmt werden können. Das [HuggingFace-Paket](https://github.com/huggingface/) bietet ein Repository für das Training vieler dieser Architekturen mit PyTorch.

## Verwendung von BERT für Textklassifikation

Schauen wir uns an, wie wir ein vortrainiertes BERT-Modell verwenden können, um unsere traditionelle Aufgabe zu lösen: die Sequenzklassifikation. Wir werden unser ursprüngliches AG-News-Dataset klassifizieren.

Zuerst laden wir die HuggingFace-Bibliothek und unser Dataset:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Da wir ein vortrainiertes BERT-Modell verwenden werden, benötigen wir einen spezifischen Tokenizer. Zunächst laden wir einen Tokenizer, der mit dem vortrainierten BERT-Modell verbunden ist.

Die HuggingFace-Bibliothek enthält ein Repository vortrainierter Modelle, die Sie einfach verwenden können, indem Sie ihre Namen als Argumente für die `from_pretrained`-Funktionen angeben. Alle erforderlichen Binärdateien für das Modell werden automatisch heruntergeladen.

Es gibt jedoch Situationen, in denen Sie Ihre eigenen Modelle laden müssen. In diesem Fall können Sie das Verzeichnis angeben, das alle relevanten Dateien enthält, einschließlich der Parameter für den Tokenizer, der `config.json`-Datei mit den Modellparametern, der Binärgewichte usw.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Das `tokenizer`-Objekt enthält die `encode`-Funktion, die direkt verwendet werden kann, um Text zu kodieren:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Dann erstellen wir Iteratoren, die wir während des Trainings verwenden, um auf die Daten zuzugreifen. Da BERT seine eigene Kodierungsfunktion verwendet, müssen wir eine Padding-Funktion ähnlich der zuvor definierten `padify`-Funktion definieren:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

In unserem Fall werden wir ein vortrainiertes BERT-Modell namens `bert-base-uncased` verwenden. Laden wir das Modell mit dem Paket `BertForSequenceClassification`. Dies stellt sicher, dass unser Modell bereits über die erforderliche Architektur für die Klassifikation verfügt, einschließlich des finalen Klassifikators. Sie werden eine Warnmeldung sehen, die besagt, dass die Gewichte des finalen Klassifikators nicht initialisiert sind und das Modell ein Pre-Training erfordert – das ist völlig in Ordnung, denn genau das werden wir jetzt tun!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Jetzt können wir mit dem Training beginnen! Da BERT bereits vortrainiert ist, möchten wir mit einer eher kleinen Lernrate starten, um die anfänglichen Gewichte nicht zu zerstören.

Die Hauptarbeit wird vom Modell `BertForSequenceClassification` erledigt. Wenn wir das Modell auf die Trainingsdaten anwenden, liefert es sowohl den Verlust als auch die Netzwerkausgabe für den Eingabe-Minibatch. Den Verlust verwenden wir für die Parameteroptimierung (`loss.backward()` führt den Backward-Pass aus), und `out` nutzen wir, um die Trainingsgenauigkeit zu berechnen, indem wir die erhaltenen Labels `labs` (berechnet mit `argmax`) mit den erwarteten `labels` vergleichen.

Um den Prozess zu kontrollieren, akkumulieren wir Verlust und Genauigkeit über mehrere Iterationen und geben sie alle `report_freq` Trainingszyklen aus.

Dieses Training wird wahrscheinlich ziemlich lange dauern, daher begrenzen wir die Anzahl der Iterationen.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Man kann sehen (besonders wenn man die Anzahl der Iterationen erhöht und lange genug wartet), dass die Klassifikation mit BERT uns eine ziemlich gute Genauigkeit liefert! Das liegt daran, dass BERT die Struktur der Sprache bereits sehr gut versteht und wir nur den finalen Klassifikator feinabstimmen müssen. Allerdings ist BERT ein großes Modell, weshalb der gesamte Trainingsprozess viel Zeit in Anspruch nimmt und erhebliche Rechenleistung erfordert! (GPU, und vorzugsweise mehr als eine).

> **Hinweis:** In unserem Beispiel haben wir eines der kleinsten vortrainierten BERT-Modelle verwendet. Es gibt größere Modelle, die wahrscheinlich bessere Ergebnisse liefern.


## Bewertung der Modellleistung

Nun können wir die Leistung unseres Modells auf dem Testdatensatz bewerten. Die Evaluierungsschleife ähnelt stark der Trainingsschleife, aber wir dürfen nicht vergessen, das Modell in den Evaluierungsmodus zu versetzen, indem wir `model.eval()` aufrufen.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Wichtiges Fazit

In dieser Einheit haben wir gesehen, wie einfach es ist, ein vortrainiertes Sprachmodell aus der **transformers**-Bibliothek zu übernehmen und es an unsere Textklassifizierungsaufgabe anzupassen. Ebenso können BERT-Modelle für die Entitätsextraktion, das Beantworten von Fragen und andere NLP-Aufgaben verwendet werden.

Transformermodelle stellen den aktuellen Stand der Technik im Bereich NLP dar, und in den meisten Fällen sollten sie die erste Lösung sein, mit der Sie experimentieren, wenn Sie benutzerdefinierte NLP-Lösungen implementieren. Dennoch ist es äußerst wichtig, die grundlegenden Prinzipien von rekurrenten neuronalen Netzwerken, die in diesem Modul besprochen wurden, zu verstehen, wenn Sie fortgeschrittene neuronale Modelle entwickeln möchten.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
